In [1]:
#packages
import tqdm
import os
import sys
import glob
import pandas as pd
import shutil 
import random

In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
import config_origins as config

In [3]:
#initialise parameter
path_dataoutput = config.path_dataoutput
path_KBF = os.path.join(path_dataoutput, 'HA')
# to avoid loosing information in case we run this code with folders containg more/differetn images, we will save the images
#into a fodler with a specific ID that you can change
id_run_KBF = '27-07-2021'
path_saveIMAGE = os.path.join(path_KBF, 'AllPictures_'+id_run_KBF)

## Step 1 - list of images paths

In [4]:
#search in any folder and create a list of img paths
#os.walk: Python method walk() generates the file names in a directory tree by walking the tree top-down or bottom-up
#to avoid saving images several times, we will do it specifically for images in the KBF folder:
li_allpictures = []
for path_ in glob.glob(os.path.join(path_KBF, 'HA*', 'KBF*')):
    print('--------------------- FOLDER: %s'%path_.split('\\')[-1])
    li_allDocuments = [os.path.join(path, name) for path, subdirs, files in os.walk(path_) for name in files]
    print('have %d documents'%len(li_allDocuments))
    li_allpictures.extend([x for x in li_allDocuments if x.endswith('.JPG')])
    print('we have now a TOTAL of %d JPG images'%len(li_allpictures))
if len(li_allpictures)!=len(set(li_allpictures)):
    print('YOU HAVE DUPLICATE IMAGE NAME')
    sys.exit()
print('We have a total of %d images'%len(li_allpictures))

--------------------- FOLDER: KBF-04-01-2021
have 501 documents
we have now a TOTAL of 163 JPG images
--------------------- FOLDER: KBF-26-07-2021
have 660 documents
we have now a TOTAL of 372 JPG images
We have a total of 372 images


## Step 2 - put all images into one folder (with all HA) & random id number

In [5]:
#folder to save images with correct ID in the ONE folder including all images from all HA
if not os.path.exists(path_saveIMAGE):
        os.makedirs(path_saveIMAGE)

In [6]:
#shuffle the images path list to have a random id (that corresponds to the image order in that list)
random.seed(0) #fix random seed for reproducibility 
#sort the list first
li_allpictures = sorted(li_allpictures)
random.shuffle(li_allpictures)
li_df = []
for i,path_img in tqdm.tqdm(enumerate(li_allpictures)):
    #copy the image into ONE folder incldgin all HA, in a CODED way for blindness
    #in one folder
    new_path_img = os.path.join(path_KBF, path_saveIMAGE, str(i)+'.jpg')
    shutil.copy(path_img, new_path_img) 
    #keep inforation about that image
    li_df.append({'initial_path_img':path_img, 'actual_path_img':new_path_img, 'imageID':i})

372it [00:42,  8.66it/s]


## Step 3 - CSV file for grading the pictures

df_imgID = pd.read_csv(os.path.join(path_KBF, path_saveIMAGE,'DF_IMAGE_INFO.csv'), sep=';')
print(df_imgID.shape)
df_imgID.head(3)

In [7]:
df_imgID = pd.DataFrame(li_df)
df_imgID['HAID'] = df_imgID['initial_path_img'].map(lambda x: x.split('\\KBF')[0].split('\\')[-1])
#add a random order for intra reliability
random.seed(1)
li_random_order = list(range(0,df_imgID.shape[0]))
random.shuffle(li_random_order)
df_imgID['random_order1'] = li_random_order
df_imgID.to_csv(os.path.join(path_KBF, path_saveIMAGE,'DF_IMAGE_INFO.csv'), index=False, sep=';')
print(df_imgID.shape)
df_imgID.head(3)

(372, 5)


,initial_path_img,actual_path_img,imageID,HAID,random_order1
0,G:\VPHI\Welfare\2- Research Projects\OFHE2.Ori...,G:\VPHI\Welfare\2- Research Projects\OFHE2.Ori...,0,HA2,146
1,G:\VPHI\Welfare\2- Research Projects\OFHE2.Ori...,G:\VPHI\Welfare\2- Research Projects\OFHE2.Ori...,1,HA2,200
2,G:\VPHI\Welfare\2- Research Projects\OFHE2.Ori...,G:\VPHI\Welfare\2- Research Projects\OFHE2.Ori...,2,HA2,202
